In [9]:
import tkinter as tk
from tkinter import messagebox
from tkinter import scrolledtext
from tkinter import ttk
from functools import partial

In [10]:
def Key_Scheduling(key):
    key_length = len(key)
    if key_length > 256:
        raise ValueError("Key too long (max length = 256)")
    S = list(range(256))
    j = 0
    for i in range(256):
        j = (j + S[i] + key[i % key_length]) % 256
        S[i], S[j] = S[j], S[i]
    return S

In [11]:
def pad_key(key):
    padded_key = bytearray(256)
    key_len = len(key)
    if key_len > 256:
        raise ValueError("Key too long (max length = 256)")
    padded_key[:key_len] = bytearray(key.encode())
    padded_key[key_len:] = bytearray(256 - key_len)
    return padded_key

In [12]:
def stream_generation(S):
    i = 0
    j = 0
    while True:
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i], S[j] = S[j], S[i]
        K = S[(S[i] + S[j]) % 256]
        yield K

In [13]:
def encrypt(plaintext, key):
    key = pad_key(key)
    S = Key_Scheduling(key)
    keystream = stream_generation(S)
    encrypted_text = ""
    for c in plaintext:
        if c.isalpha():
            if c.islower():
                c = chr((ord(c) + next(keystream) - 97) % 26 + 97)
            elif c.isupper():
                c = chr((ord(c) + next(keystream) - 65) % 26 + 65)
        elif c.isnumeric():
            c = str((int(c) + next(keystream)) % 10)
        encrypted_text += c
    return encrypted_text

In [14]:
def decrypt(ciphertext, key):
    key = pad_key(key)
    S = Key_Scheduling(key)
    keystream = stream_generation(S)
    decrypted_text = ""
    for c in ciphertext:
        if c.isalpha():
            if c.islower():
                c = chr((ord(c) - next(keystream) - 97) % 26 + 97)
            elif c.isupper():
                c = chr((ord(c) - next(keystream) - 65) % 26 + 65)
        elif c.isnumeric():
            c = str((int(c) - next(keystream)) % 10)
        decrypted_text += c
    return decrypted_text

In [16]:
class Application(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Simple Encryption/Decryption App")

        self.text_label = tk.Label(self, text="Enter Text:")
        self.text_label.grid(row=0, column=0, sticky=tk.W, padx=5, pady=5)

        self.text_entry = ttk.Entry(self, width=50)
        self.text_entry.grid(row=0, column=1, columnspan=2, padx=5, pady=5)

        self.key_label = tk.Label(self, text="Enter Key:")
        self.key_label.grid(row=1, column=0, sticky=tk.W, padx=5, pady=5)

        self.key_entry = ttk.Entry(self, width=50)
        self.key_entry.grid(row=1, column=1, columnspan=2, padx=5, pady=5)

        self.mode_label = tk.Label(self, text="Select Mode:")
        self.mode_label.grid(row=2, column=0, sticky=tk.W, padx=5, pady=5)

        self.mode_var = tk.StringVar(value="Encrypt")
        self.mode_combobox = ttk.Combobox(self, textvariable=self.mode_var, values=["Encrypt", "Decrypt"])
        self.mode_combobox.grid(row=2, column=1, padx=5, pady=5)

        self.output_label = tk.Label(self, text="Output:")
        self.output_label.grid(row=3, column=0, sticky=tk.W, padx=5, pady=5)

        self.output_text = scrolledtext.ScrolledText(self, width=50, height=10)
        self.output_text.grid(row=3, column=1, columnspan=2, padx=5, pady=5)

        self.process_button = ttk.Button(self, text="Process", command=self.process)
        self.process_button.grid(row=4, column=1, padx=5, pady=5)

    def process(self):
        mode = self.mode_var.get()
        text = self.text_entry.get()
        key = self.key_entry.get()

        if mode == "Encrypt":
            result = encrypt(text, key)
        else:
            result = decrypt(text, key)

        self.output_text.delete(1.0, tk.END)
        self.output_text.insert(tk.END, result)

In [18]:
if __name__ == "__main__":
    app = Application()
    app.mainloop()